In [3]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

from mesa.space import SingleGrid

#Importamos Random Activation ya que queremos que los agentes tomen decisiones 1 a la vez para evitar conflictos de choque
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

#Importamos random para obtener coordenadas al azar para setear nuestros dirty cells
import random

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [4]:
class OrderingAgent(Agent):
    def __init__(self,unique_id, model ):

        self.carrying_boxes =0 

    def step(self):
        #Obtenemos las posibles celdas a las que el agente tiene información
        neighborhood = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        
         #Validamos que de las posibles celdas, no haya ya un agente ahí y
        # también validamos que si haya celdas vacías y que contengan cajas
        
        empty_cells = [cell for cell in neighborhood if (self.model.grid.is_cell_empty(cell) & self.model.grid.get_type(cell[0], cell[1]) == 2)] 
        box_cells = [cell for cell in neighborhood if (self.model.grid.get_type(cell[0], cell[1]) == 2)] 

        if empty_cells :

            if box_cells:
                
                self.carrying_boxes +=1
                box_cell = self.random.choice(box_cells)
                self.model.grid.take_box(box_cell.pos[0], box_cell.pos[1] )
                self.move_to_stack()
                
            else:
                chosen_cell = self.random.choice(empty_cells)
                self.model.grid.move_agent(self, chosen_cell)
            
    def update_cells(self):
        self.carrying_boxes  +=1   

    def move_to_stack(self):
        self.model.grid.place_box(self.pos[0],self.pos[1])

In [5]:
class OrderingModel(Model):
    def __init__(self, width, height, box_quantity):
        self.box_quanitity = box_quantity
        self.grid = SingleGrid(width, height, torus = False)
        self.schedule = RandomActivation(self)

        # 0 = empty 
        # 1 = wall
        # 2 = box
        # 3 = stack

        self.type
        num_agents = 5
        stacks = 5
        id = 1

        # Assign wall type to border cells
        for x in range(self.grid.width):
            self.grid[x][0].type = 1
            self.grid[x][self.grid.height - 1].type = 1

        for y in range(self.grid.height):
            self.grid[0][y].type = 1
            self.grid[self.grid.width - 1][y].type = 1

        for i in range(num_agents):
            x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)

            while not (self.grid.is_cell_empty((x, y)) & self.get_type == 0):
                 x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)    
            
            ordering_agent = OrderingAgent(id, self)
            self.grid.place_agent(ordering_agent, (x, y))
            self.schedule.add(ordering_agent)
            id = id + 1

        for i in range(self.box_quanitity):
            x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)

            while not (self.grid.is_cell_empty((x, y)) & self.get_type == 0):
                 x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)    
            
            self.grid[x][y].type = 2
            self.grid[x][y].box_quanitity = 1

    def take_box(self,x,y):
        self.grid[x][y].type = 0
        self.box_quanitity[x][y] -=1

    def place_box(self,x,y):
        self.grid[x][y].type = 3
        self.box_quanitity[x][y] +=1
     
    def get_type(self,x,y):
        return self[x][y].type
    

In [ ]:
WIDTH = 100
HEIGHT = 100
BOX_QUANTITY = 0.9  
MAX_STEPS = 10000


start_time = time.time()

model = OrderingModel(WIDTH, HEIGHT, BOX_QUANTITY)
    
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))